In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\hassa\Anaconda3\envs\schatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY=""
PINECONE_API_ENV=""

In [3]:
#Extract data from PDF
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [6]:
extracted_pdf=load_pdf("data/")

In [8]:
#extracted_pdf

In [9]:
#Craete Text Chunks 
def text_split(extracted_pdf):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_pdf)
    
    return text_chunks

In [10]:
text_chunks= text_split(extracted_pdf)
print("length of my chunk:",len(text_chunks))

length of my chunk: 8357


In [12]:
#download embedding model
def download_embedding():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings= download_embedding()

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 373kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 67.5kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<?, ?B/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 1.18MB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 4.91MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:39<00:00, 2.33MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 24.7kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 37.3kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.54MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 87.5kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 4.39MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 995kB/s] 
modules.json: 100%|██████████| 349/349 [00:00<00:00, 116kB/s]


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [23]:
#test
query_result = embeddings.embed_query("Hello world")
print("length",len(query_result))

length 384


In [16]:
#Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY,environment=PINECONE_API_ENV)
index="v2-srh-chatbot"

#Create embeddings for each of the text chunk and store to Pinecone
docsearch= Pinecone.from_texts([t.page_content for t in text_chunks],embeddings, index_name=index)

In [17]:
#Once Index is created
docsearch=Pinecone.from_existing_index(index_name,embeddings)
query = "What are allergies?"
docs= docsearch.similarity_search(query,k=3)

print("Results",docs)

Results [Document(page_content='the itchy, scratchy nose, eyes, and throat common inallergic rhinitis .\nThe particular allergens to which a person is sensi-', metadata={}), Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n•

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

chain_type_kwargs={"prompt": PROMPT}

In [20]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [21]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [25]:
while True:
    user_input= input(f"Input Prompt:")
    result = qa({"query":user_input})
    print("Response : ", result["result"])

Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  Latent Semantic Analysis (LSA) is a statistical method used in natural language processing, information retrieval, and other fields to analyze and understand the relationships between words and concepts in a large corpus of text. LSA uses Singular Value Decomposition (SVD) to reduce the dimensionality of the word-document matrix and uncover latent topics in the corpus. The resulting topics are not just abstract labels, but rather a set of semantically related words that can be used to represent each topic.


KeyboardInterrupt: Interrupted by user